# filtered parsed tfrecord data

In [ ]:
ds  = None # tf dataset object

In [ ]:
def filter_by_target(feat, target, weight, meta):
    """
    Suppose dataset will generate tuple of structure (feature_vec, target_vec, weight_vec, meta_vec).
    Suppose target is in [0, 1, 2], we want to exclude target = 2
    """
    if target == 2:
        return False
    else:
        return True
    

ds = ds.unbatch().filter(filter_by_target)

In [ ]:
def downsample(feat, target, weight, meta):
    """
    Suppose meta[0] is sample level random seed, raning from 0~99
    """
    return tf.math.logical_and(
        tf.math.greater(tf.math.abs(tf.strings.to_number(meta[0], tf.float32)), 10.0),
        tf.math.less(tf.math.abs(tf.strings.to_number(meta[0], tf.float32)), 30.0)
    )


ds = ds.unbatch().filter(downsample)

In [ ]:
# exclude meta column from dataset, or model.fit will fail

ds = ds.map(lambda fvec, target, wgt, meta: (fvec, target, wgt))

In [ ]:
# cache tf dataset once transformation is done
tf_cache_dir = ''

cached_ds = ds.cache(tf_cache_dir).shffule()

# multi thread model prediction

Cause python interpreter has global interpreter lock, only one thread is allowed to run at the same time. Multithread is not accelerating code speed here.


In [ ]:
def run_prediction(ds,
                   models,
                   meta_list,
                   target_col_name,
                  ):
    """
    Run model prediction.
    :param ds: tf dataset object, returning tuple of (feature_vec, target, weight, meta_columns).
    :param models: list of tuple, where first element is model name, second element is loaded tf model.
    :param meta list: dataset meta columns.
    :param target_col_name: target column name.
    
    :return: dataframe of prediction result, score columns are take from first element of model list.
    """
    import multiprocessing
    
    def _predict(model, feat):
        return model.predict(feat)
    
    final_df = pd.DataFrame()
    rec_cnt = 0
    # thread number may cause process shutdown if predicting very large data
    with multiprocessing.pool.ThreadPool(8) as pool:
        for r in ds:
            rec_cnt += r[0].shape[0]
            print(f'record number: {rec_cnt}')
            
            scores = pool.starmap(_predict, [(model, r[0]) for _, model in models])
            
            metas = r[-1].numpy()
            target_vec = r[1]
            data = np.concatenate([metas] + [target_vec] + scores, axis=1)
            df = pd.DataFrame(data, columns=meta_list + [target_col_name] + [name for name, _ in models])
            final_df = pd.concat([final_df, df])
        
    for c in meta_list:
        final_df[c] = final_df[c].str.decode('UTF-8')
        
    return final_df

## single thread prediciton

In [ ]:
def run_prediction(models, ds, meta_list):
    """
    run prediction for models.
    : param models: list of tuple, first element is model name, second element is tf model
    """
    final_df = pd.DataFrame()
    for r in ds:
        scores = []
        for _, model in models:
            score = model.predict(r[0])
            scores.append(score)
        metas = r[-1].numpy()
        data = np.concatenate([metas] + scores, axis=1)
        df = pd.DataFrame(data, columns=meta_list + [f'{name}_score' for name, _ in models])
        for c in meta_list:
            df[c] = df[c].str.decode('UTF-8')
        final_df = pd.concat([final_df, df])
    return final_df

# inspect dataset elements

In [ ]:

dataset = tf.data.TFRecordDataset(filenames=filenames,
                                  compression_type='GZIP',
                                  num_parallel_reads=tf.data.experimental.AUTOTUNE,
                                 )

for e in dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(e.numpy())
    if len(example.features.feature['new_weight'].bytes_list.value[0]) == 0:
        print(example.features.feature['new_weight'])


for e in dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(e.numpy())

    for k, v in example.features.feature.items():
        if v.WhichOneof('kind') != 'float_list':
            print(k)


for k, feature in example.features.feature.items():
    print(f"feature name {k}, {feature.WhichOneof('kind')}")
    if feature.WhichOneof('kind') == 'float_list':
        print(feature.float_list.value)
    elif feature.WhichOneof('kind') == 'bytes_list':
        print(feature.bytes_list.value)
    elif feature.WhichOneof('kind') == 'int64_list':
        print(feature.int64_list.value)
    else:
        print(feature)

# tensorflow negative infinity

In [ ]:
neg_inf = tf.constant(-np.inf, shape=[1], dtype=tf.float32)

# TF log callback

In [ ]:
log_dir = os.path.join(model_log_dir, model_name)
model_log_callback = tf.keras.callbacks.TensorBoard(log_dir, histogram_freq=1)

# tf model checkpoint

In [ ]:
ck_filepath = os.path.join(expt_ckdir, "model-{epoch:02d}.ckpt")
ck_callback = ModelCheckpoint(ck_filepath, 
                              monitor='loss',
                              verbose=1,
                              save_best_only=False,
                              mode='min',
                              save_weights_only=False, 
                              save_freq='epoch')

# parse string as number

In [ ]:
ucc21 = tf.cond(
    tf.strings.length(tf.strings.strip(meta[ucc21_idx])) == 0,
    lambda: -999.0,
    lambda: tf.strings.to_number(
        tf.strings.substr(tf.strings.strip(meta[ucc21_idx]), 0, 10),
        tf.float32,
    ),
)



# parse tfrecord given meta list

In [ ]:
import pandas as pd
import numpy as np

processed_train_dataset = train_dataset.unbatch()\
                        .filter(filter_fn)\
                        .map(retarget)\
                        .map(reweight)\
                        .batch(batch_size)


df = pd.DataFrame()
for e in processed_train_dataset.take(10):
    ret = pd.DataFrame(data=np.concatenate([e[-1].numpy(), e[1].numpy()], axis=1), columns=meta_columns + ['train_tagging'])
    df = pd.concat([df, ret])





# tensorflow prediction and save as csv

In [ ]:
import os
import time
import pandas as pd
import numpy as np

model_ckpts = [] # tf model checkpoint names
tf_models = [] # loaded tf checkpoints
cached_oot_dataset = None # tfreocrd dataset, assume return (meta, feature, label) tuple
meta_columns = ['target_col_name'] # model training target column name

prediction_result_path = '' # path to save prediction result


In [ ]:
print('begin prediction')

predict_list = []
rec_size = 0


# cache oot dataset
batch_cnt = 0
for e in cached_oot_dataset:
    batch_cnt += 1
print(f'total batch cnt', batch_cnt)


# prediction
for feature, target, weight, meta in cached_oot_dataset:
    rec_size += meta.shape[0]
    print(rec_size, time.ctime())

    score_list = []
    for model in tf_models:
        score = model.predict(feature)
        # assume sf bad score is in score[0]
        if isinstance(score, list):
            score = score[0]
        score_list.append(score)
    predict_list.append(np.hstack([meta.numpy(), np.hstack(target)] + score_list))


# save prediction result
print("save prediction result to file")
score_name_list = model_ckpts
model_predict = pd.DataFrame(np.vstack(predict_list), columns=meta_columns + target_columns + score_name_list)


def decode_val(x):
    try:
        return x.decode()
    except:
        return x


for col in model_predict.columns:
    model_predict[col] = model_predict[col].apply(decode_val)
    

tgt_dir = os.path.dirname(prediction_result_path)
os.makedirs(tgt_dir, exist_ok=True)
model_predict.to_csv(prediction_result_path, index=False, sep='\x07')
print(f"prediction result saved to {prediction_result_path}")
print('=' * 120)


print('prediction columns')
print('\n'.join(model_predict.columns.to_list()))
print('=' * 120)


